In [9]:
import importlib

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
model_module = importlib.import_module("handyrl.model")
env_module = importlib.import_module("handyrl.envs.kaggle.hungry_geese")

In [11]:
e = env_module.Environment()
e.reset()

In [12]:
obs = e.observation()
obs = torch.from_numpy(obs.reshape(1, 17, 7, 11)).clone()
print(f"size: {obs.size()}, type: {obs.dtype}")

size: torch.Size([1, 17, 7, 11]), type: torch.float32


In [13]:
e.obs_list

[[{'action': 'NORTH',
   'reward': 0,
   'info': {},
   'observation': {'remainingOverageTime': 60,
    'step': 0,
    'geese': [[16], [72], [10], [38]],
    'food': [67, 6],
    'index': 0},
   'status': 'ACTIVE'},
  {'action': 'NORTH',
   'reward': 0,
   'info': {},
   'observation': {'remainingOverageTime': 60, 'index': 1},
   'status': 'ACTIVE'},
  {'action': 'NORTH',
   'reward': 0,
   'info': {},
   'observation': {'remainingOverageTime': 60, 'index': 2},
   'status': 'ACTIVE'},
  {'action': 'NORTH',
   'reward': 0,
   'info': {},
   'observation': {'remainingOverageTime': 60, 'index': 3},
   'status': 'ACTIVE'}]]

In [14]:
input_ = torch.randn(2, 17, 7, 11)
print(f"size: {input_.size()}, type: {input_.dtype}")

size: torch.Size([2, 17, 7, 11]), type: torch.float32


In [15]:
net = env_module.GeeseNetIMO(e)

In [16]:
out = net(obs)
print(f"size: {out.size()}, type: {out.dtype}")

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
out

In [ ]:
# pytorch_total_params
params = sum(p.numel() for p in net.parameters())
print(f"{params:,}")

In [ ]:
# pytorch_total_params (trainable)
params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"{params:,}")

In [17]:
net

GeeseNetIMO(
  (encoder): GeeseEncoder(
    (conv0): TorusConv2d(
      (conv): Conv2d(17, 32, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (blocks): ModuleList(
    (0): GeeseBlock(
      (attention): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
      )
    )
    (1): GeeseBlock(
      (attention): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
      )
    )
    (2): GeeseBlock(
      (attention): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
      )
    )
    (3): GeeseBlock(
      (attention): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
      )
    )
    (4): GeeseBlock(
      (attention): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=64, out_features=64,